# Advanced Retrieval with LangChain

In the following notebook, we'll explore various methods of advanced retrieval using LangChain!

We'll touch on:

- Naive Retrieval
- Best-Matching 25 (BM25)
- Multi-Query Retrieval
- Parent-Document Retrieval
- Contextual Compression (a.k.a. Rerank)
- Ensemble Retrieval
- Semantic chunking

We'll also discuss how these methods impact performance on our set of documents with a simple RAG chain.

There will be two breakout rooms:

- 🤝 Breakout Room Part #1
  - Task 1: Getting Dependencies!
  - Task 2: Data Collection and Preparation
  - Task 3: Setting Up QDrant!
  - Task 4-10: Retrieval Strategies
- 🤝 Breakout Room Part #2
  - Activity: Evaluate with Ragas

# 🤝 Breakout Room Part #1

## Task 1: Getting Dependencies!

We're going to need a few specific LangChain community packages, like OpenAI (for our [LLM](https://platform.openai.com/docs/models) and [Embedding Model](https://platform.openai.com/docs/guides/embeddings)) and Cohere (for our [Reranker](https://cohere.com/rerank)).

In [186]:
!pip install -qU langchain langchain-openai langchain-cohere rank_bm25

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.0.1 requires dill<0.3.9,>=0.3.0, but you have dill 0.3.9 which is incompatible.
ragas 0.1.20 requires langchain-core<0.3, but you have langchain-core 0.3.7 which is incompatible.
grpcio-status 1.66.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.5 which is incompatible.


In [379]:
!pip install -qU langsmith

We're also going to be leveraging [Qdrant's](https://qdrant.tech/documentation/frameworks/langchain/) (pronounced "Quadrant") VectorDB in "memory" mode (so we can leverage it locally in our colab environment).

In [188]:
!pip install -qU qdrant-client

We'll also provide our OpenAI key, as well as our Cohere API key.

In [190]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")


Enter your OpenAI API Key: ········


In [191]:
os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")



Cohere API Key: ········


## Task 2: Data Collection and Preparation

We'll be using some reviews from the 4 movies in the John Wick franchise today to explore the different retrieval strategies.

These were obtained from IMDB, and are available in the [AIM Data Repository](https://github.com/AI-Maker-Space/DataRepository).

### Data Collection

We can simply `wget` these from GitHub.

You could use any review data you wanted in this step - just be careful to make sure your metadata is aligned with your choice.

In [194]:
# !wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw1.csv -O john_wick_1.csv
# !wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw2.csv -O john_wick_2.csv
# !wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw3.csv -O john_wick_3.csv
# !wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw4.csv -O john_wick_4.csv

### Data Preparation

We want to make sure all our documents have the relevant metadata for the various retrieval strategies we're going to be applying today.

- Self-Query: Wants as much metadata as we can provide
- Time-weighted: Wants temporal data

> NOTE: While we're creating a temporal relationship based on when these movies came out for illustrative purposes, it needs to be clear that the "time-weighting" in the Time-weighted Retriever is based on when the document was *accessed* last - not when it was created.

In [196]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from datetime import datetime, timedelta

documents = []

for i in range(1, 5):
  loader = CSVLoader(
      file_path=f"john_wick_{i}.csv",
      metadata_columns=["Review_Date", "Review_Title", "Review_Url", "Author", "Rating"]
  )

  movie_docs = loader.load()
  for doc in movie_docs:

    # Add the "Movie Title" (John Wick 1, 2, ...)
    doc.metadata["Movie_Title"] = f"John Wick {i}"

    # convert "Rating" to an `int`, if no rating is provided - assume 0 rating
    doc.metadata["Rating"] = int(doc.metadata["Rating"]) if doc.metadata["Rating"] else 0

    # newer movies have a more recent "last_accessed_at"
    doc.metadata["last_accessed_at"] = datetime.now() - timedelta(days=4-i)

  documents.extend(movie_docs)

Let's look at an example document to see if everything worked as expected!

In [198]:
documents[0]

Document(metadata={'source': 'john_wick_1.csv', 'row': 0, 'Review_Date': '6 May 2015', 'Review_Title': ' Kinetic, concise, and stylish; John Wick kicks ass.\n', 'Review_Url': '/review/rw3233896/?ref_=tt_urv', 'Author': 'lnvicta', 'Rating': 8, 'Movie_Title': 'John Wick 1', 'last_accessed_at': datetime.datetime(2024, 9, 27, 23, 12, 59, 674708)}, page_content=": 0\nReview: The best way I can describe John Wick is to picture Taken but instead of Liam Neeson it's Keanu Reeves and instead of his daughter it's his dog. That's essentially the plot of the movie. John Wick (Reeves) is out to seek revenge on the people who took something he loved from him. It's a beautifully simple premise for an action movie - when action movies get convoluted, they get bad i.e. A Good Day to Die Hard. John Wick gives the viewers what they want: Awesome action, stylish stunts, kinetic chaos, and a relatable hero to tie it all together. John Wick succeeds in its simplicity.")

## Task 3: Setting up QDrant!

Now that we have our documents, let's create a QDrant VectorStore with the collection name "JohnWick".

We'll leverage OpenAI's [`text-embedding-3-small`](https://openai.com/blog/new-embedding-models-and-api-updates) because it's a very powerful (and low-cost) embedding model.

> NOTE: We'll be creating additional vectorstores where necessary, but this pattern is still extremely useful.

In [200]:
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = Qdrant.from_documents(
    documents,
    embeddings,
    location=":memory:",
    collection_name="JohnWick"
)

## Task 4: Naive RAG Chain

Since we're focusing on the "R" in RAG today - we'll create our Retriever first.

### R - Retrieval

This naive retriever will simply look at each review as a document, and use cosine-similarity to fetch the 10 most relevant documents.

> NOTE: We're choosing `10` as our `k` here to provide enough documents for our reranking process later

In [203]:
naive_retriever = vectorstore.as_retriever(search_kwargs={"k" : 10})

### A - Augmented

We're going to go with a standard prompt for our simple RAG chain today! Nothing fancy here, we want this to mostly be about the Retrieval process.

In [205]:
from langchain_core.prompts import ChatPromptTemplate

RAG_TEMPLATE = """\
You are a helpful and kind assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

### G - Generation

We're going to leverage `gpt-3.5-turbo` as our LLM today, as - again - we want this to largely be about the Retrieval process.

In [207]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI()

### LCEL RAG Chain

We're going to use LCEL to construct our chain.

> NOTE: This chain will be exactly the same across the various examples with the exception of our Retriever!

In [209]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

naive_retrieval_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | naive_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's see how this simple chain does on a few different prompts.

> NOTE: You might think that we've cherry picked prompts that showcase the individual skill of each of the retrieval strategies - you'd be correct!

In [211]:
naive_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Yes, people generally liked John Wick based on the positive reviews provided.'

In [212]:
naive_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

"Yes, there is a review with a rating of 10. Here is the URL to that review: '/review/rw4854296/?ref_=tt_urv'"

In [213]:
naive_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In John Wick, an ex-hitman comes out of retirement to seek vengeance against the gangsters who killed his dog and took everything from him. This leads to a series of violent confrontations and action-packed sequences as John Wick navigates a world of assassins and criminals.'

Overall, this is not bad! Let's see if we can make it better!

## Task 5: Best-Matching 25 (BM25) Retriever

Taking a step back in time - [BM25](https://www.nowpublishers.com/article/Details/INR-019) is based on [Bag-Of-Words](https://en.wikipedia.org/wiki/Bag-of-words_model) which is a sparse representation of text.

In essence, it's a way to compare how similar two pieces of text are based on the words they both contain.

This retriever is very straightforward to set-up! Let's see it happen down below!


In [216]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(documents)

We'll construct the same chain - only changing the retriever.

In [218]:
naive_retrieval_chain = (
    {"context": itemgetter("question") | bm25_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at the responses!

In [220]:
naive_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

"Opinions on John Wick seem to be divided based on the reviews provided. Some people really enjoyed the movie and found it to be a must-see for action fans, while others found it to be lacking in plot and substance. So, it's safe to say that not everyone liked John Wick."

In [221]:
naive_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

"I don't know."

In [222]:
naive_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'John Wick is an action movie with beautifully choreographed scenes and emotional setup. It stars Keanu Reeves and is highly recommended for action movie lovers.'

It's not clear that this is better or worse - but the `I don't know` isn't great!

## Task 6: Contextual Compression (Using Reranking)

Contextual Compression is a fairly straightforward idea: We want to "compress" our retrieved context into just the most useful bits.

There are a few ways we can achieve this - but we're going to look at a specific example called reranking.

The basic idea here is this:

- We retrieve lots of documents that are very likely related to our query vector
- We "compress" those documents into a smaller set of *more* related documents using a reranking algorithm.

We'll be leveraging Cohere's Rerank model for our reranker today!

All we need to do is the following:

- Create a basic retriever
- Create a compressor (reranker, in this case)

That's it!

Let's see it in the code below!

In [225]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor = CohereRerank(model="rerank-english-v3.0")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=naive_retriever
)

Let's create our chain again, and see how this does!

In [227]:
contextual_compression_retrieval_chain = (
    {"context": itemgetter("question") | compression_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [228]:
contextual_compression_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Yes, people generally liked John Wick based on the positive reviews and high ratings it received.'

In [229]:
contextual_compression_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

"Yes, there is a review with a rating of 10. Here is the URL to that review:\n- Review URL: '/review/rw4854296/?ref_=tt_urv'"

In [230]:
contextual_compression_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

"In John Wick 2, after resolving his issues with the Russian mafia, John Wick is asked by Santino D'Antonio to kill his sister Gianna D'Antonio in Rome. When John completes the assignment, Santino puts a seven-million dollar contract on him, leading to professional killers coming after him. Wick promises to kill Santino, who is not protected by his marker anymore."

We'll need to rely on something like Ragas to help us get a better sense of how this is performing overall - but it "feels" better!

## Task 7: Multi-Query Retriever

Typically in RAG we have a single query - the one provided by the user.

What if we had....more than one query!

In essence, a Multi-Query Retriever works by:

1. Taking the original user query and creating `n` number of new user queries using an LLM.
2. Retrieving documents for each query.
3. Using all unique retrieved documents as context

So, how is it to set-up? Not bad! Let's see it down below!



In [233]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=naive_retriever, llm=chat_model
)

In [234]:
multi_query_retrieval_chain = (
    {"context": itemgetter("question") | multi_query_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [235]:
multi_query_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Yes, people generally liked John Wick based on the reviews provided.'

In [236]:
multi_query_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'No, there are no reviews with a rating of 10 for the movie "John Wick 4."'

In [237]:
multi_query_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

"In John Wick, the main character, played by Keanu Reeves, is a retired assassin who is forced back into action when his car is stolen, leading to a lot of carnage. He is also called upon to pay off an old debt by helping take over the Assassin's Guild in various locations by killing many assassins."

## Task 8: Parent Document Retriever

A "small-to-big" strategy - the Parent Document Retriever works based on a simple strategy:

1. Each un-split "document" will be designated as a "parent document" (You could use larger chunks of document as well, but our data format allows us to consider the overall document as the parent chunk)
2. Store those "parent documents" in a memory store (not a VectorStore)
3. We will chunk each of those documents into smaller documents, and associate them with their respective parents, and store those in a VectorStore. We'll call those "child chunks".
4. When we query our Retriever, we will do a similarity search comparing our query vector to the "child chunks".
5. Instead of returning the "child chunks", we'll return their associated "parent chunks".

Okay, maybe that was a few steps - but the basic idea is this:

- Search for small documents
- Return big documents

The intuition is that we're likely to find the most relevant information by limiting the amount of semantic information that is encoded in each embedding vector - but we're likely to miss relevant surrounding context if we only use that information.

Let's start by creating our "parent documents" and defining a `RecursiveCharacterTextSplitter`.

In [239]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models

parent_docs = documents
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

We'll need to set up a new QDrant vectorstore - and we'll use another useful pattern to do so!

> NOTE: We are manually defining our embedding dimension, you'll need to change this if you're using a different embedding model.

In [241]:
client = QdrantClient(location=":memory:")

client.create_collection(
    collection_name="full_documents",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
)

parent_document_vectorstore = Qdrant(
    collection_name="full_documents", embeddings=OpenAIEmbeddings(model="text-embedding-3-small"), client=client
)

Now we can create our `InMemoryStore` that will hold our "parent documents" - and build our retriever!

In [243]:
store = InMemoryStore()

parent_document_retriever = ParentDocumentRetriever(
    vectorstore = parent_document_vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

By default, this is empty as we haven't added any documents - let's add some now!

In [245]:
parent_document_retriever.add_documents(parent_docs, ids=None)

We'll create the same chain we did before - but substitute our new `parent_document_retriever`.

In [247]:
parent_document_retrieval_chain = (
    {"context": itemgetter("question") | parent_document_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's give it a whirl!

In [249]:
parent_document_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Based on the reviews provided, opinions on John Wick seem to be divided. Some people love the series, while others have strong negative opinions about it.'

In [250]:
parent_document_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10 for the movie "John Wick 3". The URL to that review is \'/review/rw4854296/?ref_=tt_urv\'.'

In [251]:
parent_document_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

"In John Wick, Keanu Reeves plays the character John Wick, a retired assassin who comes out of retirement after someone kills his dog. In the sequel, John Wick 2, he is called on to pay off an old debt by helping take over the Assassin's Guild by traveling to Italy, Canada, and Manhattan and killing many assassins."

Overall, the performance *seems* largely the same. We can leverage a tool like [Ragas]() to more effectively answer the question about the performance.

## Task 9: Ensemble Retriever

In brief, an Ensemble Retriever simply takes 2, or more, retrievers and combines their retrieved documents based on a rank-fusion algorithm.

In this case - we're using the [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) algorithm.

Setting it up is as easy as providing a list of our desired retrievers - and the weights for each retriever.

In [254]:
from langchain.retrievers import EnsembleRetriever

retriever_list = [bm25_retriever, naive_retriever, parent_document_retriever, compression_retriever, multi_query_retriever]
equal_weighting = [1/len(retriever_list)] * len(retriever_list)

ensemble_retriever = EnsembleRetriever(
    retrievers=retriever_list, weights=equal_weighting
)

We'll pack *all* of these retrievers together in an ensemble.

In [256]:
ensemble_retrieval_chain = (
    {"context": itemgetter("question") | ensemble_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at our results!

In [258]:
ensemble_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Yes, people generally liked John Wick based on the reviews provided.'

In [259]:
ensemble_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10 for "John Wick 3". Here is the URL to that review: /review/rw4854296/?ref_=tt_urv'

In [260]:
ensemble_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In John Wick, an ex-hitman comes out of retirement to seek vengeance against the gangsters who killed his dog and took everything from him. The story involves a lot of violence, shootouts, and breathtaking fights as John Wick faces off against various enemies.'

## Task 10: Semantic Chunking

While this is not a retrieval method - it *is* an effective way of increasing retrieval performance on corpora that have clean semantic breaks in them.

Essentially, Semantic Chunking is implemented by:

1. Embedding all sentences in the corpus.
2. Combining or splitting sequences of sentences based on their semantic similarity based on a number of [possible thresholding methods](https://python.langchain.com/docs/how_to/semantic-chunker/):
  - `percentile`
  - `standard_deviation`
  - `interquartile`
  - `gradient`
3. Each sequence of related sentences is kept as a document!

Let's see how to implement this!

In [262]:
!pip install -qU langchain_experimental

We'll use the `percentile` thresholding method for this example which will:

Calculate all distances between sentences, and then break apart sequences of setences that exceed a given percentile among all distances.

In [264]:
from langchain_experimental.text_splitter import SemanticChunker

semantic_chunker = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile"
)

Now we can split our documents.

In [266]:
semantic_documents = semantic_chunker.split_documents(documents)

Let's create a new vector store.

In [268]:
semantic_vectorstore = Qdrant.from_documents(
    semantic_documents,
    embeddings,
    location=":memory:",
    collection_name="JohnWickSemantic"
)

We'll use naive retrieval for this example.

In [270]:
semantic_retriever = semantic_vectorstore.as_retriever(search_kwargs={"k" : 10})

Finally we can create our classic chain!

In [272]:
semantic_retrieval_chain = (
    {"context": itemgetter("question") | semantic_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

And view the results!

In [274]:
ensemble_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Yes, people generally liked John Wick based on the reviews provided.'

In [275]:
ensemble_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

"Yes, there is a review with a rating of 10. Here is the URL to that review:\n- Review_URL: '/review/rw4854296/?ref_=tt_urv'"

In [276]:
ensemble_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'John Wick is a movie about an ex-hitman who comes out of retirement to seek vengeance against the gangsters who killed his dog and took everything from him. The movie is known for its violent and gripping story with plenty of action, shootouts, and breathtaking fights.'

# 🤝 Breakout Room Part #2

#### 🏗️ Activity #1

Your task is to evaluate the various Retriever methods against eachother.

You are expected to:

1. Create a "golden dataset"
 - Use Synthetic Data Generation (powered by Ragas, or otherwise) to create this dataset
2. Evaluate each retriever with *retriever specific* Ragas metrics
 - Semantic Chunking is not considered a retriever method and will not be required for marks, but you may find it useful to do a "semantic chunking on" vs. "semantic chunking off" comparision between them
3. Compile these in a list and write a small paragraph about which is best for this particular data and why.

Your analysis should factor in:
  - Cost
  - Latency
  - Performance

> NOTE: This is **NOT** required to be completed in class. Please spend time in your breakout rooms creating a plan before moving on to writing code.

##### HINTS:

- LangSmith provides detailed information about latency and cost.

In [280]:
### YOUR CODE HERE

In [359]:
import os
from uuid import uuid4
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")
unique_id = uuid4().hex[0:8]

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"Advanced RAG - {unique_id}"

LangChain API Key: ········


In [ ]:
# os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")


In [ ]:
# os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

In [282]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from datetime import datetime, timedelta

documents = []

for i in range(1, 5):
  loader = CSVLoader(
      file_path=f"john_wick_{i}.csv",
      metadata_columns=["Review_Date", "Review_Title", "Review_Url", "Author", "Rating"]
  )

  movie_docs = loader.load()
  for doc in movie_docs:

    # Add the "Movie Title" (John Wick 1, 2, ...)
    doc.metadata["Movie_Title"] = f"John Wick {i}"

    # convert "Rating" to an `int`, if no rating is provided - assume 0 rating
    doc.metadata["Rating"] = int(doc.metadata["Rating"]) if doc.metadata["Rating"] else 0

    # newer movies have a more recent "last_accessed_at"
    doc.metadata["last_accessed_at"] = datetime.now() - timedelta(days=4-i)

  documents.extend(movie_docs)

In [283]:
documents[0]


Document(metadata={'source': 'john_wick_1.csv', 'row': 0, 'Review_Date': '6 May 2015', 'Review_Title': ' Kinetic, concise, and stylish; John Wick kicks ass.\n', 'Review_Url': '/review/rw3233896/?ref_=tt_urv', 'Author': 'lnvicta', 'Rating': 8, 'Movie_Title': 'John Wick 1', 'last_accessed_at': datetime.datetime(2024, 9, 27, 23, 15, 6, 833281)}, page_content=": 0\nReview: The best way I can describe John Wick is to picture Taken but instead of Liam Neeson it's Keanu Reeves and instead of his daughter it's his dog. That's essentially the plot of the movie. John Wick (Reeves) is out to seek revenge on the people who took something he loved from him. It's a beautifully simple premise for an action movie - when action movies get convoluted, they get bad i.e. A Good Day to Die Hard. John Wick gives the viewers what they want: Awesome action, stylish stunts, kinetic chaos, and a relatable hero to tie it all together. John Wick succeeds in its simplicity.")

In [284]:

from langchain_community.vectorstores  import Qdrant
from langchain_openai import OpenAIEmbeddings



embeddings = OpenAIEmbeddings(model="text-embedding-3-small")


vectorstore = Qdrant.from_documents(
    documents,
    embeddings,
    location=":memory:",
    collection_name="JohnWick"
)

In [285]:
naive_retriever = vectorstore.as_retriever(search_kwargs={"k" : 10})


In [286]:
from langchain_core.prompts import ChatPromptTemplate

RAG_TEMPLATE = """\
You are a helpful and kind assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

In [287]:

from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(seed=42)

In [288]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

naive_retrieval_chain = (
    {"context": itemgetter("question") | naive_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [289]:
# !pip install -qU pymupdf ragas

In [290]:
# !pip install langchain_google_vertexai

In [291]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

generator_llm = ChatOpenAI(model="gpt-4o-mini")
critic_llm = ChatOpenAI(model="gpt-4o")
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

distributions = {
    simple: 0.5,
    multi_context: 0.4,
    reasoning: 0.1
}

/opt/anaconda3/lib/python3.12/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in _VertexAIBase has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in _VertexAICommon has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/ragas/metrics/__init__.py:8: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain.pydantic_v1 import BaseModel`
with:

In [292]:
testset = generator.generate_with_langchain_docs(documents, 20, distributions, with_debugging_logs=False)


embedding nodes: 0it [00:00, ?it/s]

ExceptionInRunner: The runner thread which was running the jobs raised an exeception. Read the traceback above to debug it. You can also pass `raise_exceptions=False` incase you want to show only a warning message instead.

In [ ]:
testset

In [295]:
for i, doc in enumerate(documents):
    try:
        testset = generator.generate_with_langchain_docs(
            [doc],  # Process one document at a time
            1, 
            distributions, 
            with_debugging_logs=True, 
            raise_exceptions=True
        )
        print(f"Document {i} processed successfully.")
    except Exception as e:
        print(f"Error with document {i}: {str(e)}")

embedding nodes: 0it [00:00, ?it/s]

Error with document 0: The runner thread which was running the jobs raised an exeception. Read the traceback above to debug it. You can also pass `raise_exceptions=False` incase you want to show only a warning message instead.


embedding nodes: 0it [00:00, ?it/s]

Error with document 1: The runner thread which was running the jobs raised an exeception. Read the traceback above to debug it. You can also pass `raise_exceptions=False` incase you want to show only a warning message instead.


embedding nodes: 0it [00:00, ?it/s]

...

Error with document 97: The runner thread which was running the jobs raised an exeception. Read the traceback above to debug it. You can also pass `raise_exceptions=False` incase you want to show only a warning message instead.


embedding nodes: 0it [00:00, ?it/s]

Error with document 98: The runner thread which was running the jobs raised an exeception. Read the traceback above to debug it. You can also pass `raise_exceptions=False` incase you want to show only a warning message instead.


embedding nodes: 0it [00:00, ?it/s]

Error with document 99: The runner thread which was running the jobs raised an exeception. Read the traceback above to debug it. You can also pass `raise_exceptions=False` incase you want to show only a warning message instead.


In [ ]:
testset = generator.generate_with_langchain_docs(
    documents, 
    20, 
    distributions, 
    with_debugging_logs=True, 
    raise_exceptions=True, 
    is_async=False  # Disable async execution to simplify debugging
)

In [305]:
!pip install --upgrade ragas langchain langchain_community

  Using cached langchain_core-0.2.41-py3-none-any.whl.metadata (6.2 kB)
INFO: pip is looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain-0.3.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain-0.3.0-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain-0.2.16-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_text_splitters-0.2.4-py3-none-any.whl.metadata (2.3 kB)
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_community-0.3.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached langchain_community-0.3.0-py3-none-any.whl.metadata (2.8 kB)
  Using cached langchain_community-0.2.17-py3-none-any.whl.metadata (2.7 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
INFO: pip is looking at multiple versions of langchain-op

In [307]:
testset = generator.generate_with_langchain_docs(documents, 20, distributions, with_debugging_logs=False)


embedding nodes: 0it [00:00, ?it/s]

ExceptionInRunner: The runner thread which was running the jobs raised an exeception. Read the traceback above to debug it. You can also pass `raise_exceptions=False` incase you want to show only a warning message instead.

In [353]:

# import os
# from uuid import uuid4


# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

# os.environ["LANGCHAIN_PROJECT"] = f"AIE4 - A14 - BM25 {uuid4().hex[0:8]}"
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# print(f"Your langsmith key is: {os.environ['LANGCHAIN_PROJECT']}")

LangChain API Key: ········


Your langsmith key is: AIE4 - A14 - BM25 bbaa6be0


In [371]:
import os
from uuid import uuid4
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
COHERE_API_KEY = os.environ.get("COHERE_API_KEY")

unique_id = uuid4().hex[0:8]


os.environ["LANGCHAIN_PROJECT"] = f"Advanced RAG - {unique_id}"

LangChain API Key: ········


In [311]:
response = naive_retrieval_chain.invoke({"question" : "Who dies in John Wick?"})['response']
print(response)
print(response.content)  

content='In "John Wick," John\'s dog dies.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 3908, 'total_tokens': 3918, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-2150a815-cf77-48bc-b8aa-e6c1f8fed216-0' usage_metadata={'input_tokens': 3908, 'output_tokens': 10, 'total_tokens': 3918}
In "John Wick," John's dog dies.


In [313]:
documents[0]
print(len(documents))
content_1 = documents[0].page_content
print(len(content_1))
lengths = [len(doc.page_content) for doc in documents]
average_length = sum(lengths) / len(lengths) if lengths else 0
min_length = min(lengths) if lengths else 0
max_length = max(lengths) if lengths else 0

print(f"Average Content Length: {average_length}")
print(f"Minimum Content Length: {min_length}")
print(f"Maximum Content Length: {max_length}")

100
599
Average Content Length: 533.9
Minimum Content Length: 29
Maximum Content Length: 2440


In [315]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

generator_llm = ChatOpenAI(model="gpt-3.5-turbo")
critic_llm = ChatOpenAI(model="gpt-4o-mini")
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

distributions = {
    simple: 0.5,
    multi_context: 0.4,
    reasoning: 0.1
}

In [317]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
import pickle

chunk_size = 1000
chunk_overlap = 100
file_path = 'ragas_testset.pkl'

# load an existing ragas testset
def load_ragas_testset_if_exists():
    if os.path.exists(file_path):
        try:
            with open(file_path, 'rb') as f:
                ragas_state = pickle.load(f)
            print(f"Ragas testset loaded from {file_path}")
            return ragas_state
        except Exception as e:
            print(f"Error loading ragas testset: {e}")
            return None
    else:
        print(f"No existing ragas tesetset found at {file_path}")
        return None

# Save the ragas testset
def save_ragas_testset(testset):

    try:
        with open(file_path, 'wb') as f:
            pickle.dump(testset, f)
        print(f"Ragas testset saved to {file_path}")
    except Exception as e:
        print(f"Error saving ragas testset: {e}")


# create questions
def create_questions_for_ragas(documents, num_questions=1):
    generator_llm = ChatOpenAI(model="gpt-3.5-turbo")
    critic_llm = ChatOpenAI(model="gpt-4o-mini")
    embeddings = OpenAIEmbeddings()

    generator = TestsetGenerator.from_langchain(
        generator_llm,
        critic_llm,
        embeddings
    )
    distributions = {
        simple: 0.5,
        multi_context: 0.4,
        reasoning: 0.1
    }
    
    testset = generator.generate_with_langchain_docs(documents, num_questions, distributions, with_debugging_logs=False)
    save_ragas_testset(testset)
    return testset

In [323]:
create_questions = False
ragas_testset = None
num_questions = 40
if create_questions:
    ragas_testset = create_questions_for_ragas(documents, num_questions)
else:
    ragas_testset = load_ragas_testset_if_exists()
if ragas_testset:
    ragas_testset.to_pandas()
else:
    print("No RAGAS testset found - need to create questions")

Ragas testset loaded from ragas_testset.pkl


In [391]:
testset = ragas_testset.to_pandas()


In [331]:
testset

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What makes John Wick stand out as a favorite r...,[: 22\nReview: John Wick is one of my favourit...,John Wick stands out as a favorite recent year...,simple,"[{'source': 'john_wick_2.csv', 'row': 22, 'Rev...",True
1,What are some examples of classic internationa...,[: 11\nReview: JOHN WICK is a rare example of ...,Some examples of classic international films s...,simple,"[{'source': 'john_wick_1.csv', 'row': 11, 'Rev...",True
2,What was the surprise hit movie starring Keanu...,"[: 6\nReview: In 2014, a Keanu Reeves revenge ...",John Wick,simple,"[{'source': 'john_wick_2.csv', 'row': 6, 'Revi...",True
3,Who does John Wick face as he is called upon b...,[: 5\nReview: Iosef's uncle still has John Wic...,"John Wick faces deadly assassins, numerous kil...",simple,"[{'source': 'john_wick_2.csv', 'row': 5, 'Revi...",True
4,"What is the genre and main actor of the film ""...",[: 21\nReview: John Wick is an action film wit...,"The genre of the film 'John Wick' is action, a...",simple,"[{'source': 'john_wick_1.csv', 'row': 21, 'Rev...",True


In [ ]:
from langsmith import Client

client = Client()

# commented the following code because has already run once and will mess up Langsmith

dataset_name = "John Wicks Dataset 3"

dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Questions about John Wicks movie reviews"
)

In [383]:
# from langsmith import Client

# client = Client()

# dataset_name = "AI Regulations Paragraph Chunks v3"

# dataset = client.create_dataset(
#     dataset_name=dataset_name,
#     description="Questions about AI regulations"
# )

In [393]:
type(testset)

pandas.core.frame.DataFrame

In [395]:
test_df = testset

In [397]:

test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()

In [399]:

answers = []
contexts = []

for question in test_questions:
  response = naive_retrieval_chain.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])

In [401]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

In [403]:
response_dataset[0]


{'question': 'What makes John Wick stand out as a favorite recent years action movie?',
 'answer': "John Wick stands out as a favorite recent years action movie due to its slickness, the brilliance in its action sequences, Keanu Reeves' performance as the titular character, the Tarantino-esque dialog mixed with the swagger of action stars from the 70s, the well-paced and meticulously choreographed action scenes, and the overall fun and intense experience it provides viewers. The film is praised for its straightforward plot, clear motives, practical stunt work, lack of shaky cams or quick cuts, and the ability to deliver on what action fans want.",
 'contexts': [": 9\nReview: At first glance, John Wick sounds like a terrible film on paper but with the slickness of Keanu Reeves' performance as the titular character and the sheer brilliance in its action sequences, this marks the best action film of the year and one of the absolute best in the past decade. Following a brutal home invasion

In [405]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

In [407]:

from langsmith import traceable

def run_evaluation(retrieval_chain):
    answers = []
    contexts = []

    for question in test_questions:
        response = retrieval_chain.invoke({"question" : question})
        answers.append(response["response"].content)
        contexts.append([context.page_content for context in response["context"]])
    response_dataset = Dataset.from_dict({
        "question" : test_questions,
        "answer" : answers,
        "contexts" : contexts,
        "ground_truth" : test_groundtruths
    })
    return evaluate(response_dataset, metrics)

In [409]:
unique_id = uuid4().hex[0:8]
@traceable(
        run_type="llm",
        name="OpenAI Call Decorator",
        project_name=f"Naive Retrieval - {unique_id}"
)
def run_naive():
    return run_evaluation(naive_retrieval_chain)

naive_results = run_naive()

Evaluating:   0%|          | 0/25 [00:00<?, ?it/s]

In [411]:
naive_results

{'faithfulness': 0.9333, 'answer_relevancy': 0.9664, 'context_recall': 1.0000, 'context_precision': 0.9599, 'answer_correctness': 0.8131}

In [413]:

from langchain_community.retrievers import BM25Retriever
bm25_retriever = BM25Retriever.from_documents(documents)

In [415]:
bm25_retrieval_chain = (
    {"context": itemgetter("question") | bm25_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [417]:
unique_id = uuid4().hex[0:8]
@traceable(
        run_type="llm",
        name="OpenAI Call Decorator",
        project_name=f"BM25 Retrieval - {unique_id}"
)
def run_bm25():
    return run_evaluation(bm25_retrieval_chain)

bm25_results = run_bm25()

Evaluating:   0%|          | 0/25 [00:00<?, ?it/s]

In [419]:
bm25_results

{'faithfulness': 0.9600, 'answer_relevancy': 0.9525, 'context_recall': 1.0000, 'context_precision': 0.9833, 'answer_correctness': 0.7621}

In [421]:
import getpass
os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

Cohere API Key: ········


In [423]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor = CohereRerank(model="rerank-english-v3.0")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=naive_retriever
)

contextual_compression_retrieval_chain = (
    {"context": itemgetter("question") | compression_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [425]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
compressor = LLMChainExtractor.from_llm(chat_model)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=naive_retriever
)
compression_retrieval_chain = (
    {"context": itemgetter("question") | compression_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)
unique_id = uuid4().hex[0:8]
@traceable(
        run_type="llm",
        name="OpenAI Call Decorator",
        project_name=f"Context Compression Retrieval - {unique_id}"
)
def run_compression():
    return run_evaluation(compression_retrieval_chain)

compression_results = run_compression()

Evaluating:   0%|          | 0/25 [00:00<?, ?it/s]

In [427]:
compression_results

{'faithfulness': 0.8000, 'answer_relevancy': 0.9747, 'context_recall': 1.0000, 'context_precision': 0.7640, 'answer_correctness': 0.8572}

In [429]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=naive_retriever, llm=chat_model
)

In [431]:
multi_query_retrieval_chain = (
    {"context": itemgetter("question") | multi_query_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [433]:
unique_id = uuid4().hex[0:8]
@traceable(
        run_type="llm",
        name="OpenAI Call Decorator",
        project_name=f"Multiquery Retrieval - {unique_id}"
)
def run_multi_query():
    return run_evaluation(multi_query_retrieval_chain)

multi_query_results = run_multi_query()

Evaluating:   0%|          | 0/25 [00:00<?, ?it/s]

In [468]:
multi_query_results

{'faithfulness': 0.9000, 'answer_relevancy': 0.9735, 'context_recall': 1.0000, 'context_precision': 0.9187, 'answer_correctness': 0.8590}

In [435]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models

parent_docs = documents
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

In [436]:
client = QdrantClient(location=":memory:")

client.create_collection(
    collection_name="full_documents",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
)

parent_document_vectorstore = Qdrant(
    collection_name="full_documents", 
    embeddings=OpenAIEmbeddings(model="text-embedding-3-small"), 
    client=client
)

In [437]:
store = InMemoryStore()

parent_document_retriever = ParentDocumentRetriever(
    vectorstore = parent_document_vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

In [438]:
parent_document_retriever.add_documents(parent_docs, ids=None)


In [439]:
parent_document_retrieval_chain = (
    {"context": itemgetter("question") | parent_document_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [440]:
unique_id = uuid4().hex[0:8]
@traceable(
        run_type="llm",
        name="OpenAI Call Decorator",
        project_name=f"Parent Document Retrieval - {unique_id}"
)
def run_parent_query():
    return run_evaluation(parent_document_retrieval_chain)

parent_results = run_parent_query()

Evaluating:   0%|          | 0/25 [00:00<?, ?it/s]

In [470]:
parent_results

{'faithfulness': 0.8300, 'answer_relevancy': 0.9619, 'context_recall': 0.8667, 'context_precision': 1.0000, 'answer_correctness': 0.9760}

In [441]:
from langchain.retrievers import EnsembleRetriever

retriever_list = [bm25_retriever, naive_retriever, parent_document_retriever, multi_query_retriever]
equal_weighting = [1/len(retriever_list)] * len(retriever_list)

ensemble_retriever = EnsembleRetriever(
    retrievers=retriever_list, weights=equal_weighting
)

In [442]:
ensemble_retrieval_chain = (
    {"context": itemgetter("question") | ensemble_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [443]:
unique_id = uuid4().hex[0:8]
@traceable(
        run_type="llm",
        name="OpenAI Call Decorator",
        project_name=f"Ensemble Retrieval - {unique_id}"
)
def run_ensemble_query():
    return run_evaluation(ensemble_retrieval_chain)

ensemble_results = run_ensemble_query()

Evaluating:   0%|          | 0/25 [00:00<?, ?it/s]

In [472]:
ensemble_results

{'faithfulness': 1.0000, 'answer_relevancy': 0.9690, 'context_recall': 1.0000, 'context_precision': 0.9556, 'answer_correctness': 0.8916}

In [444]:
from langchain_experimental.text_splitter import SemanticChunker

semantic_chunker = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile"
)
semantic_documents = semantic_chunker.split_documents(documents)

In [445]:
semantic_vectorstore = Qdrant.from_documents(
    semantic_documents,
    embeddings,
    location=":memory:",
    collection_name="JohnWickSemantic"
)
semantic_retriever = semantic_vectorstore.as_retriever(search_kwargs={"k" : 10})

In [446]:
semantic_retrieval_chain = (
    {"context": itemgetter("question") | semantic_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [447]:
unique_id = uuid4().hex[0:8]
@traceable(
        run_type="llm",
        name="OpenAI Call Decorator",
        project_name=f"Semantic Naive Retrieval - {unique_id}"
)
def run_semantic_naive():
    return run_evaluation(semantic_retrieval_chain)
semantic_results = run_semantic_naive()

Evaluating:   0%|          | 0/25 [00:00<?, ?it/s]

In [476]:
semantic_results

{'faithfulness': 0.9000, 'answer_relevancy': 0.9762, 'context_recall': 0.9333, 'context_precision': 0.7698, 'answer_correctness': 0.8782}